# Post-processing CSV

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime


import cv2
import numpy as np
import matplotlib.pylab as plt

%matplotlib inline

In [3]:
import sys
sys.path.append("../common/")

In [4]:
from data_utils import get_scalers, LABELS, TRAIN_IMAGE_IDS
from image_utils import get_image_data
from submission_utils import submission_iterator, write_shp_from_polygons, write_shp_from_csv, write_shp_from_mask
from submission_utils import get_scaled_polygons, get_polygons, compute_label_image, get_data_csv, get_image_ids
from submission_utils import rewrite_submission
from visu_utils import plt_st, display_img_1b

In [5]:
for i, l in enumerate(LABELS):
    print i, l

0 None
1 Buildings
2 Misc. Manmade structures
3 Road
4 Track
5 Trees
6 Crops
7 Waterway
8 Standing water
9 Vehicle Large
10 Vehicle Small


Compare submission's predictions raw and post-processed on trained data with ground truth

In [6]:
from np_metrics import score

Get common image ids from submission file and training image ids

In [7]:
def write_shp_pred_pp(image_id, csv_file):
    data_csv = get_data_csv(image_id, csv_file)
    assert data_csv is not None, "No data_csv"
    write_shp_from_csv("../results/shp/%s_new.shp" % image_id, data_csv)
    polygons = get_scaled_polygons(data_csv)
    y_pred_raw = compute_label_image(image_id, polygons)
    y_pred_pp = mask_postprocessing(y_pred_raw, class_pp_func_list)
    write_shp_from_mask("../results/shp/%s_pp.shp" % image_id, image_id, y_pred_pp)

In [8]:
def compare_true_pred_pp(image_id, csv_file, write_shp=True, visualize_labels=()):
    y_true = get_image_data(image_id, 'label')
    data_csv = get_data_csv(image_id, csv_file)
    assert data_csv is not None, "No data_csv"
    if write_shp:
        write_shp_from_csv("../results/shp/%s_new.shp" % image_id, data_csv)
    polygons = get_scaled_polygons(data_csv)    
    y_pred_raw = compute_label_image(image_id, polygons)
    
    h = min(y_true.shape[0], y_pred_raw.shape[0])
    w = min(y_true.shape[1], y_pred_raw.shape[1])
    s1 = score(y_true[:h,:w,:], y_pred_raw[:h,:w,:])    
    print "True vs Pred Raw: ", s1
    
    polygons = shape_postprocessing(polygons, class_shape_pp_func_list)
    y_pred_pp = compute_label_image(image_id, polygons)
    y_pred_pp = mask_postprocessing(y_pred_pp, class_pp_func_list)
    
    h = min(y_true.shape[0], y_pred_pp.shape[0])
    w = min(y_true.shape[1], y_pred_pp.shape[1])
    s2 = score(y_true[:h,:w,:], y_pred_pp[:h,:w,:])
    print "True vs Pred PP: ", s2
    if write_shp:
        write_shp_from_mask("../results/shp/%s_pp.shp" % image_id, image_id, y_pred_pp)
        
        
        
    if len(visualize_labels) > 0:
        ll = len(visualize_labels)
        for i in visualize_labels:
            if i % 2 == 0:
                plt_st(12, 6)
            plt.subplot(131)
            display_img_1b(y_true[:,:,i])
            plt.title("Truth")
            plt.subplot(132)
            display_img_1b(y_pred_raw[:,:,i])
            plt.title("Raw")
            plt.subplot(133)
            display_img_1b(y_pred_pp[:,:,i])
            plt.title("PP")
        
        
        
    return s1, s2

In [10]:
#CSV_FILE_NEW = "../results/sub_mix_20170304_2329.csv"
#CSV_FILE_NEW = "../results/submission_2017-03-05-23-26.csv"
CSV_FILE_NEW = "../results/union_201703071611.csv"
CSV_FILE_TRAIN = "../results/sub_model_1_20170304_1826_nan.csv"


In [11]:
from postprocessing_utils import mask_postprocessing
from postprocessing_utils import trees_postprocessing, buildings_postprocessing, roads_postprocessing, standing_water_postprocessing
from postprocessing_utils import shape_postprocessing, roads_shape_postprocessing
from postprocessing_utils import buildings_shape_postprocessing


class_pp_func_list = {
    1: buildings_postprocessing, 
#     3: roads_postprocessing,
    5: trees_postprocessing, 
    8: standing_water_postprocessing, 
}

class_shape_pp_func_list = {
    #1: buildings_shape_postprocessing, # Hand-made labeling is forbidden :(
    3: roads_shape_postprocessing
}

Check on all training data:

In [12]:
train_ids = TRAIN_IMAGE_IDS

In [13]:
total_s1 = 0
total_s2 = 0
for image_id in train_ids:
    print "--", image_id
    s1, s2 = compare_true_pred_pp(image_id, CSV_FILE_TRAIN, write_shp=False)
    total_s1 += s1
    total_s2 += s2

total_s1 *= 1.0 / len(train_ids)
total_s2 *= 1.0 / len(train_ids)
print "Final score: ", total_s1, total_s2

-- 6040_2_2
True vs Pred Raw:  0.124523
True vs Pred PP:  0.124417
-- 6120_2_2
True vs Pred Raw:  0.301466
True vs Pred PP:  0.305632
-- 6120_2_0
True vs Pred Raw:  0.332018
True vs Pred PP:  0.336076
-- 6090_2_0
True vs Pred Raw:  0.14743
True vs Pred PP:  0.15652
-- 6040_1_3
True vs Pred Raw:  0.137345
True vs Pred PP:  0.136793
-- 6040_1_0
True vs Pred Raw:  0.145944
True vs Pred PP:  0.143905
-- 6100_1_3
True vs Pred Raw:  0.349125
True vs Pred PP:  0.364399
-- 6010_4_2
True vs Pred Raw:  0.131629
True vs Pred PP:  0.132173
-- 6110_4_0
True vs Pred Raw:  0.398946
True vs Pred PP:  0.406851
-- 6140_3_1
True vs Pred Raw:  0.335387
True vs Pred PP:  0.345627
-- 6110_1_2
True vs Pred Raw:  0.233097
True vs Pred PP:  0.240389
-- 6100_2_3
True vs Pred Raw:  0.293448
True vs Pred PP:  0.306018
-- 6150_2_3
True vs Pred Raw:  0.2014
True vs Pred PP:  0.204052
-- 6160_2_1
True vs Pred Raw:  0.139019
True vs Pred PP:  0.139813
-- 6140_1_2
True vs Pred Raw:  0.243752
True vs Pred PP:  0.249001

Standing water

-- 6040_2_2
True vs Pred Raw:  0.124523
True vs Pred PP:  0.124523
-- 6120_2_2
True vs Pred Raw:  0.301466
True vs Pred PP:  0.302711
-- 6120_2_0
True vs Pred Raw:  0.332018
True vs Pred PP:  0.332018
-- 6090_2_0
True vs Pred Raw:  0.14743
True vs Pred PP:  0.155803
-- 6040_1_3
True vs Pred Raw:  0.137345
True vs Pred PP:  0.137345
-- 6040_1_0
True vs Pred Raw:  0.145944
True vs Pred PP:  0.145944
-- 6100_1_3
True vs Pred Raw:  0.349125
True vs Pred PP:  0.352157
-- 6010_4_2
True vs Pred Raw:  0.131629
True vs Pred PP:  0.131629
-- 6110_4_0
True vs Pred Raw:  0.398946
True vs Pred PP:  0.399476
-- 6140_3_1
True vs Pred Raw:  0.335387
True vs Pred PP:  0.338185
-- 6110_1_2
True vs Pred Raw:  0.233097
True vs Pred PP:  0.23354
-- 6100_2_3
True vs Pred Raw:  0.293448
True vs Pred PP:  0.295865
-- 6150_2_3
True vs Pred Raw:  0.2014
True vs Pred PP:  0.2014
-- 6160_2_1
True vs Pred Raw:  0.139019
True vs Pred PP:  0.139019
-- 6140_1_2
True vs Pred Raw:  0.243752
True vs Pred PP:  0.243999
-- 6110_3_1
True vs Pred Raw:  0.355562
True vs Pred PP:  0.358455
-- 6010_4_4
True vs Pred Raw:  0.120205
True vs Pred PP:  0.120205
-- 6170_2_4
True vs Pred Raw:  0.220104
True vs Pred PP:  0.219598
-- 6170_4_1
True vs Pred Raw:  0.117173
True vs Pred PP:  0.117173
-- 6170_0_4
True vs Pred Raw:  0.1282
True vs Pred PP:  0.1282
-- 6060_2_3
True vs Pred Raw:  0.183337
True vs Pred PP:  0.183337
-- 6070_2_3
True vs Pred Raw:  0.213583
True vs Pred PP:  0.212168
-- 6010_1_2
True vs Pred Raw:  0.119722
True vs Pred PP:  0.119722
-- 6040_4_4
True vs Pred Raw:  0.121834
True vs Pred PP:  0.121834
-- 6100_2_2
True vs Pred Raw:  0.256762
True vs Pred PP:  0.256735
Final score:  0.214040441513 0.214841640592



Buildings + Roads1 + Trees + Water

-- 6040_2_2
True vs Pred Raw:  0.124523
True vs Pred PP:  0.124417
-- 6120_2_2
True vs Pred Raw:  0.301466
True vs Pred PP:  0.300468
-- 6120_2_0
True vs Pred Raw:  0.332018
True vs Pred PP:  0.333829
-- 6090_2_0
True vs Pred Raw:  0.14743
True vs Pred PP:  0.15652
-- 6040_1_3
True vs Pred Raw:  0.137345
True vs Pred PP:  0.136793
-- 6040_1_0
True vs Pred Raw:  0.145944
True vs Pred PP:  0.143904
-- 6100_1_3
True vs Pred Raw:  0.349125
True vs Pred PP:  0.36002
-- 6010_4_2
True vs Pred Raw:  0.131629
True vs Pred PP:  0.132173
-- 6110_4_0
True vs Pred Raw:  0.398946
True vs Pred PP:  0.406208
-- 6140_3_1
True vs Pred Raw:  0.335387
True vs Pred PP:  0.343092
-- 6110_1_2
True vs Pred Raw:  0.233097
True vs Pred PP:  0.23561
-- 6100_2_3
True vs Pred Raw:  0.293448
True vs Pred PP:  0.297894
-- 6150_2_3
True vs Pred Raw:  0.2014
True vs Pred PP:  0.204052
-- 6160_2_1
True vs Pred Raw:  0.139019
True vs Pred PP:  0.1398
-- 6140_1_2
True vs Pred Raw:  0.243752
True vs Pred PP:  0.246068
-- 6110_3_1
True vs Pred Raw:  0.355562
True vs Pred PP:  0.36527
-- 6010_4_4
True vs Pred Raw:  0.120205
True vs Pred PP:  0.120364
-- 6170_2_4
True vs Pred Raw:  0.220104
True vs Pred PP:  0.218764
-- 6170_4_1
True vs Pred Raw:  0.117173
True vs Pred PP:  0.117373
-- 6170_0_4
True vs Pred Raw:  0.1282
True vs Pred PP:  0.12805
-- 6060_2_3
True vs Pred Raw:  0.183337
True vs Pred PP:  0.185514
-- 6070_2_3
True vs Pred Raw:  0.213583
True vs Pred PP:  0.21341
-- 6010_1_2
True vs Pred Raw:  0.119722
True vs Pred PP:  0.120306
-- 6040_4_4
True vs Pred Raw:  0.121834
True vs Pred PP:  0.120114
-- 6100_2_2
True vs Pred Raw:  0.256762
True vs Pred PP:  0.262434
Final score:  0.214040441513 0.21649792999

Save new submission

In [13]:
submission_file = '../results/submission_' + str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv'
rewrite_submission(CSV_FILE_NEW, submission_file, class_pp_func_list, class_shape_pp_func_list)

-- 6120_2_4 10 0
-- 6120_2_3 10 1


Self-intersection at or near point 3193.5167785234898 1985.6040268456377


-- 6120_2_1 10 2
-- 6180_2_4 10 3


Ring Self-intersection at or near point 2247 3101


-- 6180_2_1 10 4


...

-- 6130_1_3 10 425


Ring Self-intersection at or near point 3203 1515


-- 6130_1_2 10 426
-- 6130_1_1 10 427


Ring Self-intersection at or near point 1020 2057


-- 6130_1_0 10 428


In [14]:
import pandas as pd

In [15]:
df1 = pd.read_csv(CSV_FILE_NEW)
df2 = pd.read_csv(submission_file)

In [16]:
assert (df1['ImageId'] == df2['ImageId']).all(), "WTF"
assert (df1['ClassType'] == df2['ClassType']).all(), "WTF"

In [17]:
# assert (df1['MultipolygonWKT'] == df2['MultipolygonWKT']).all(), "WTF"

## Visual data analysis

### Generate a 6XYZ image and SHP

In [19]:
import gdal
from visu_utils import scale_percentile

def generate_complete_image(image_id):
    filename = os.path.join("..", "results", "visu", '%s.tif' % image_id)
    if os.path.exists(filename):
        print "Image is already created. Remove it"
        os.remove(filename)        

    tile_size = (1024, 1024)
    n, m = 5, 5
    complete_image = np.zeros((5*tile_size[0], 5*tile_size[1]), dtype=np.uint16)
    counter = 0
    for i in range(n):
        ys = i*tile_size[1]
        ye = ys + tile_size[1]
        for j in range(m):
            xs = j*tile_size[0]
            xe = xs + tile_size[0]
            _id = image_id + "_%i_%i" % (i, j)
            img = get_image_data(_id, 'pan')
            img = cv2.resize(img, dsize=tile_size)    
            complete_image[ys:ye, xs:xe] = img

    complete_image_8u = scale_percentile(complete_image, 0.1, 99.9)             
    # Initialize output file:
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(filename, complete_image_8u.shape[1], complete_image_8u.shape[0], 1, gdal.GDT_Byte)
    assert dataset is not None, "Failed to create a file"

    band = dataset.GetRasterBand(1)
    band.WriteArray(complete_image_8u, xoff=0, yoff=0)

    dataset = None

In [24]:
import os
from submission_utils import get_data_csv
from shapely.wkt import loads
from shapely.affinity import scale, translate

def get_transformed_polygons(data_csv, x, y, w, h):
    out = {}
    image_id = data_csv[0][0]
    xs, ys = get_scalers(image_id, h, w)
    for poly_info in data_csv:
        if "MULTIPOLYGON" not in poly_info[2][:20]:
            continue
        polygons = loads(poly_info[2])
        new_polygons = scale(polygons, xfact=xs, yfact=ys, origin=(0, 0, 0))        
        new_polygons = translate(new_polygons, xoff=x, yoff=y)
        out[int(poly_info[1])] = new_polygons
    return out

from collections import defaultdict

def generate_complete_shp_from_csv(image_id, csv_filename, suffix=''):
    
    filename = os.path.join("..", "results", "visu", '%s_%s.shp' % (image_id, suffix))
    if os.path.exists(filename):
        print "Image is already created. Remove it"
        os.remove(filename)        
        
    tile_size = (1024, 1024)
    n, m = 5, 5
    all_polygons = defaultdict(list)
    counter = 0
    for i in range(n):
        y = i*tile_size[1]
        for j in range(m):
            x = j*tile_size[0]
            _id = image_id + "_%i_%i" % (i, j)
            print "--", _id
            data_csv = get_data_csv(_id, csv_filename)        
            if data_csv is None:
                continue
            polygons = get_transformed_polygons(data_csv, x, y, tile_size[0], tile_size[1])
            for k in polygons:
                all_polygons[k].append(polygons[k])
    write_shp_from_polygons(filename, image_id, all_polygons)


In [14]:
CSV_FILE_LAST = submission_file
CSV_FILE_LAST[-20:-4]

In [22]:
# get_image_ids(CSV_FILE_LAST)

In [61]:
image_id = "6110"
# image_id = "6120"
# image_id = "6070"
# image_id = "6080"
# image_id = "6050"

# image_id = "6140"

In [46]:
generate_complete_image(image_id)

In [62]:
generate_complete_shp_from_csv(image_id, CSV_FILE_LAST, CSV_FILE_LAST[-20:-4])

-- 6110_0_0
-- 6110_0_1
-- 6110_0_2
-- 6110_0_3
-- 6110_0_4
-- 6110_1_0
-- 6110_1_1
-- 6110_1_2
-- 6110_1_3
-- 6110_1_4
-- 6110_2_0
-- 6110_2_1
-- 6110_2_2
-- 6110_2_3
-- 6110_2_4
-- 6110_3_0
-- 6110_3_1
-- 6110_3_2
-- 6110_3_3
-- 6110_3_4
-- 6110_4_0
-- 6110_4_1
-- 6110_4_2
-- 6110_4_3
-- 6110_4_4
Written succesfully file :  ../results/visu/6110_2017-03-07-02-17.shp


### Generate a SHP with all prediction for all image_id

In [25]:
import os
from collections import defaultdict
from data_utils import ALL_IMAGE_IDS
all_ids = sorted(np.unique([iid[:-4] for iid in ALL_IMAGE_IDS]).tolist())

def generate_all_shp_from_csv(csv_filename, suffix=''):
    filename = os.path.join("..", "results", "visu", 'all_predictions_%s.shp' % (suffix))
    assert not os.path.exists(filename), "Image is already created. Remove it manually"
        
    n_tiles = 18
    nx = 5
    ny = int(np.ceil(n_tiles / float(nx)))
    
    tile_size = (850, 850)
    all_polygons = defaultdict(list)
    count = 0
    for data_csv in submission_iterator(csv_filename):  
        image_id = data_csv[0][0]
        count += 1
        print "--", image_id
        index = all_ids.index(image_id[:4])
        i, j = (int(image_id[7]), int(image_id[5]))
        
        xi = index % nx * (5*tile_size[0]) + i*tile_size[0]
        yi = int(np.floor(index * 1.0 / nx)) * (5*tile_size[1]) + j*tile_size[1]        
        #print xi, yi
        polygons = get_transformed_polygons(data_csv, xi, yi, tile_size[0], tile_size[1])
        for k in polygons:
            all_polygons[k].append(polygons[k])
    write_shp_from_polygons(filename, "ALL", all_polygons)    

In [27]:
generate_all_shp_from_csv("../results/union_201703071611_pair_merge.csv", "union_201703071611_pair_merge")

-- 6120_2_4
-- 6120_2_3
-- 6120_2_1
-- 6180_2_4
-- 6180_2_1
-- 6180_2_0
-- 6180_2_3
-- 6180_2_2
-- 6180_0_3
-- 6180_0_2
-- 6180_0_1
-- 6180_0_0
-- 6180_0_4
-- 6080_4_4
-- 6080_4_2
-- 6080_4_3
-- 6080_4_0
-- 6080_4_1
-- 6090_4_1
-- 6090_4_0
-- 6090_4_3
-- 6090_4_2
-- 6090_4_4
-- 6180_4_4
-- 6180_4_3
-- 6180_4_2
-- 6180_4_1
-- 6180_4_0
-- 6160_3_2
-- 6160_3_3
-- 6160_3_0
-- 6160_3_1
-- 6160_3_4
-- 6080_2_4
-- 6080_2_0
-- 6080_2_1
-- 6080_2_2
-- 6080_2_3
-- 6080_0_2
-- 6080_0_3
-- 6080_0_0
-- 6080_0_1
-- 6080_0_4
-- 6010_0_4
-- 6010_0_1
-- 6010_0_0
-- 6010_0_3
-- 6010_0_2
-- 6010_2_3
-- 6010_2_2
-- 6010_2_1
-- 6010_2_0
-- 6010_2_4
-- 6010_4_1
-- 6170_3_4
-- 6010_4_3
-- 6170_3_1
-- 6170_3_0
-- 6170_3_3
-- 6170_3_2
-- 6170_1_3
-- 6170_1_2
-- 6170_1_1
-- 6170_1_0
-- 6170_1_4
-- 6130_4_2
-- 6130_4_3
-- 6130_4_0
-- 6130_4_1
-- 6130_4_4
-- 6150_3_3
-- 6150_3_2
-- 6150_3_1
-- 6150_3_0
-- 6150_3_4
-- 6130_2_0
-- 6130_2_1
-- 6130_2_2
-- 6130_2_3
-- 6130_2_4
-- 6130_0_4
-- 6130_0_2
-- 6130_0_3
-- 6